The most popular optimization method in machine learning are is ADAM: *"Adam: A Method for Stochastic Optimization"* https://arxiv.org/abs/1412.6980. This method use only first derivative and the step is different for all of the parameters and it is changed in the way that combine the RMSprop and momentum method. I've found that better for your case work method which has different adaptive approach we calculate the estimation of the second derivative using the changes of derivative from two steps.

Below you can find definition of the Eva structure used in optimization, the code come from the QuantumCircuit library: https://github.com/Adgnitio/QuantumCircuits.jl/blob/main/src/QML/Optimization.jl.

In [1]:

"Evolutionary Alpha optimization algoritm"
mutable struct Eva{T <: AbstractFloat}
#mutable struct Eva{T}
    α::Vector{T}
    df::Vector{T}
    x::Vector{T}
    val::T
    doUpdate::Bool
    lastSingularCount::Int

    Eva{T}(alfa::T, len) where {T <: AbstractFloat} = new(fill(alfa, len), zeros(T, len), zeros(T, len), 0, false, 0)
    #Eva{T}(alfa::T, len) where {T} = new(fill(alfa, len), zeros(T, len), zeros(T, len), false)
end
function (e::Eva)(x, val, df; argsArePeriodic=false, debug=false, useBigValInc=false)
    ϵ = 1e-20
    ϵ2 = 1e-10
    BigValIncreas = 1.1

    #println("==========================")
    #println("Val: $val, e.val: $(e.val)")
    maxH = 0.0
    maxDiffX = 0.0
    maxα = 0.0
    debug && val > e.val * BigValIncreas && e.doUpdate && useBigValInc && println("Big value increase: val: $val, e.val: $(e.val), max(α): $(maximum(e.α))")
    for i in 1:length(x)
        notsingular = true
        if e.doUpdate
            #i == 1 && @show i, df[i], e.df[i], x[i], e.x[i], e.α[i]

            diffx = x[i] - e.x[i]
            maxDiffX = max(maxDiffX, abs(diffx))
            if e.df[i] * df[i] < 0 && val > e.val  && abs(e.df[i]) > ϵ2 && abs(df[i]) > ϵ2
                # we are in singular point (there is no partial derivate)
                notsingular = false
                α = e.α[i]
            elseif e.lastSingularCount > 0
                α = e.α[i]
                e.lastSingularCount -= 1
            elseif abs(diffx) > ϵ
                H = (df[i] - e.df[i]) / diffx
                #@show H
                α = 1.0 / (abs(H) + ϵ)
                e.α[i] = α

                maxH = max(maxH, abs(H))
            else
                # the d'' = 0, so we are in sedle point, try to pertube some
                α = 1.2 * e.α[i]
                #α = e.α[i]
            end
        else
            α = e.α[i]
        end
        #@show α, x[i], df[i]
        #debug && !notsingular && println("$i singular: x[i]: $(x[i]), e.x[i]: $(e.x[i]), singular: df[i]: $(df[i]), e.df[i]: $(e.df[i])")

        # There was big increas of value
        if val > e.val * BigValIncreas && e.doUpdate && useBigValInc
            #debug && println("$i big diff: x[i]: $(x[i]), e.x[i]: $(e.x[i]), singular: df[i]: $(df[i]), e.df[i]: $(e.df[i])")
            e.α[i] = min(e.α[i] / 2, 0.1)
            x[i] = e.x[i] - e.α[i] * e.df[i]
            val = e.val
        else
            e.df[i] = df[i]
            if notsingular
                maxα = max(maxα, α)
                if argsArePeriodic
                    if x[i] > PeriodThreshold || x[i] < -PeriodThreshold
                        x[i] = x[i] % Period
                    end
                end
                e.x[i] = x[i]
                x[i] = x[i] - α * df[i]
            else
                ix = x[i]
                x[i] = (x[i] + e.x[i]) / 2
                e.x[i] = ix
                e.lastSingularCount = 10
                e.α[i] = min(0.1, α/2)
            end
        end
    end
    e.val = val
    if e.doUpdate
        return (maxH, maxα, maxDiffX)
    else
        e.doUpdate = true
        return (nothing, maxα, maxDiffX)
    end
end